In [ ]:
import sqlite3
import networkx as nx
from pyvis.network import Network

In [ ]:
conn = sqlite3.connect('/GeneGraphDB/data/genegraph.db')
conn.row_factory = sqlite3.Row

In [ ]:
def get_tables(conn):
    tables = []
    
    cur = conn.cursor()
    cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
    for table_tup in cur.fetchall():
        tables.append(table_tup[0])
    cur.close()
    
    return tables

tables = get_tables(conn)
list(sorted(tables))

In [ ]:
for table in tables:
    print(f"TABLE: {table}\n")
    cur = conn.cursor()
    cur.execute(f"SELECT * FROM {table} LIMIT 1;")
    for row in cur.fetchall():
        for key in row.keys():
            print(f"{key}: {row[key]}")
        print("\n")
    cur.close()
    
    print("******\n\n")

In [ ]:
cur = conn.cursor()
cur.execute(f"SELECT * FROM clusters WHERE p30 is 'c2b5fea0b52ea0ac15'")
for idx, row in enumerate(cur.fetchall()):
    print(idx)
    for key in row.keys():
        print(f"{key}: {row[key]}")
    print("\n")
cur.close()

In [ ]:
def get_targets(bait_hash):
    tgt_hashes = set()
    cur = conn.cursor()
    cur.execute(f"SELECT * FROM prot2protwindow WHERE p1hash is '{bait_hash}' OR p2hash is '{bait_hash}'")
    for row in cur.fetchall():
        for key in row.keys():
            tgt_hashes.add(row[key])
    cur.close()
    
    tgt_hashes.remove(bait_hash)
    return list(sorted(tgt_hashes))

bait_hash = "693388b375dbd274e1"
print("BAIT", bait_hash)
tgt_hashes = get_targets(bait_hash)
print("TARGETS")
display(tgt_hashes)

tgt_hash = tgt_hashes[0]

In [ ]:
def get_cluster(p100):
    cur = conn.cursor()
    cur.execute(f"SELECT * FROM clusters WHERE p100 is '{p100}'")
    all_rows = cur.fetchall()
    assert len(all_rows) == 1, f"Found {len(all_rows)} clusters for p100 {p100}"
    cur.close()
    row = all_rows[0]
    return row

def get_window_neighbors(pid):
    cur = conn.cursor()
    cur.execute(f"SELECT * FROM prot2protwindow WHERE p1hash is '{pid}' OR p2hash is '{pid}'")
    rows = list(cur.fetchall())
    cur.close()
    return rows

def get_entire_p30_cluster(cluster_id):
    cur = conn.cursor()
    cur.execute(f"SELECT * FROM clusters WHERE p30 is '{cluster_id}'")
    rows = list(cur.fetchall())
    cur.close()
    return rows

BLACK="#000000"
WHITE="#FFFFFF"
RED="#FF0000"
GREEN="#00FF00"
BLUE="#0000FF"
YELLOW="#FFFF00"
CYAN="#00FFFF"
MAGENTA="#FF00FF"
PURPLE="#A020F0"


G = nx.Graph()
# add target node
G.add_node(tgt_hash, type='p100', size=10, color=CYAN)
# add bait node
G.add_node(bait_hash, type='p100', size=10, color=MAGENTA)


node_to_cousins = {
    tgt_hash: set(),
    bait_hash: set()
}


# add target cluster, bait cluster
for node, p100_color in ((tgt_hash, BLUE), (bait_hash, PURPLE)):
    cluster_row = get_cluster(node)
    p90 = cluster_row['p90'] + "_p90"
    p30 = cluster_row['p30'] + "_p30"
    G.add_node(p90, size=25, color=YELLOW)
    G.add_edge(p90, node, type="p90_cluster")
    G.add_node(p30, size=50, color=RED)
    G.add_edge(p30, p90, type="p30_cluster")
    
    # add other p90s from cluster
    for cluster_row in get_entire_p30_cluster(cluster_row['p30']):
        print(f"found p100 for {node}")
        p100 = cluster_row['p100']
        p90 = cluster_row['p90'] + "_p90"
        p30 = cluster_row['p30'] + "_p30"
        G.add_node(p100, size=10, color=p100_color)
        G.add_node(p90, size=25, color=YELLOW)
        G.add_edge(p90, p100, type="p90_cluster")
        G.add_node(p30, size=50, color=RED)
        G.add_edge(p30, p90, type="p30_cluster")
        node_to_cousins[node].add(p100)

for cousin in node_to_cousins[tgt_hash]:
    cousin_neighbors = get_targets(cousin)
    for neighbor in cousin_neighbors:
        if neighbor in node_to_cousins[bait_hash]:
            print("found connection")
            G.add_edge(cousin, neighbor)

# p100_neighbors = get_targets(p100)        
        
# add edges

# add target node
G.add_node(tgt_hash, type='p100', size=10, color=CYAN)
# add bait node
G.add_node(bait_hash, type='p100', size=10, color=MAGENTA)

# G.add_edge(tgt_hash, bait_hash)





# for idx, row in enumerate(get_window_neighbors(tgt_hash)):
#     for row_key in ["p1hash", "p2hash"]:
#         G.add_node(row[row_key], type='p100', size=10, color="#00ff1e")
#         cluster_row = get_cluster(row[row_key])
#         G.add_node(cluster_row['p90'], type='p90', size=20, color="#367440")
#         G.add_edge(cluster_row['p90'], cluster_row['p100'], type='p90_cluster')
#         G.add_node(cluster_row['p30'], type='p30', size=30)
#         G.add_edge(cluster_row['p30'], cluster_row['p90'], type='p30_cluster', color="#8722E1")
#     G.add_edge(row['p1hash'], row['p2hash'], type='5kb_window')




In [ ]:
net = Network(notebook=True)
net.from_nx(G)
net.show('./pyvis_test.html')

In [ ]:
related_clusters = [n for n in G if G.nodes[n]['type']=='p30']
for cluster in related_clusters[0:3]:
    cluster_rows = get_entire_p30_cluster(cluster)
    print(len(cluster_rows))
    for cluster_row in cluster_rows:
        G.add_node(cluster_row['p90'], type='p90', size=20, color="#367440")
        G.add_edge(cluster_row['p90'], cluster_row['p100'], type='p90_cluster')
        G.add_node(cluster_row['p30'], type='p30', size=30)
        G.add_edge(cluster_row['p30'], cluster_row['p90'], type='p30_cluster', color="#8722E1")

In [ ]:
net = Network()
net.from_nx(G)
net.show('./pyvis_test.html')

In [ ]:
net.get_node('e1912a410e1beb69fe')